In [3]:
import  random
import numpy as np
import time
import pickle

In [2]:
import tensorflow as tf
from tensorflow.python.saved_model import tag_constants

In [4]:
X_subTr = np.loadtxt('XTr.dat')
y_subCdTr = (np.loadtxt('yTr.dat')).reshape((-1,1))

In [5]:
X_subTr.shape

(42039, 16)

In [6]:
y_subCdTr.shape

(42039, 1)

In [8]:
X_subTr = np.loadtxt('XTr.dat')# (42039, 16) 
X_subV = np.loadtxt('XV.dat') # (10935, 16) 
y_subCdTr = (np.loadtxt('yTr.dat')).reshape((-1,1)) # (42039, 1) 
y_subCdV = (np.loadtxt('yV.dat')).reshape((-1,1)) # (10935, 1) 
dy_subCdTr = np.loadtxt('dyTr.dat') # (42039, 16) 
X_transTr = np.loadtxt('X_transTr.dat') # (4120, 10) 
X_transV = np.loadtxt('X_transV.dat') # (787, 10) 
y_transCdTr = (np.loadtxt('y_transCdTr.dat')).reshape((-1,1)) # (4120, 1) 
y_transCdV = (np.loadtxt('y_transV.dat')).reshape((-1,1)) # (787, 1) 
dy_transCdTr = np.loadtxt('dy_transCdTr.dat') # (4120, 10)
print(X_subTr.shape,X_subV.shape,y_subCdTr.shape,y_subCdV.shape,dy_subCdTr.shape,X_transTr.shape,X_transV.shape,y_transCdTr.shape,y_transCdV.shape,dy_transCdTr.shape)

(42039, 16) (10935, 16) (42039, 1) (10935, 1) (42039, 16) (4120, 10) (787, 10) (4120, 1) (787, 1) (4120, 10)


In [9]:
# Read dimensions of the data
nSubTr = X_subTr.shape[0] # 42039
nSubV = X_subV.shape[0] #  10935
dimSub = X_subV.shape[1] # 16
nTransTr = X_transTr.shape[0] # 4120
nTransV = X_transV.shape[0] # 787
dimTrans = X_transV.shape[1] # 10
print(nSubTr,nSubV,dimSub,nTransTr,nTransV,dimTrans)

42039 10935 16 4120 787 10


In [10]:
# Define the new design space
XTr = np.zeros((nTransTr+nSubTr,dimTrans + dimSub - 2))
print(XTr.shape)
XTr[:nTransTr,:dimTrans-2] = X_transTr[:,:dimTrans-2]
print(XTr.shape)
XTr[nTransTr:,dimTrans-2:] = X_subTr
print(XTr.shape)
XTr[:nTransTr,-2:] = X_transTr[:,-2:]
print(XTr.shape)
XV = np.zeros((nTransV+nSubV,dimTrans + dimSub - 2))
print(XV.shape)
XV[:nTransV,:dimTrans-2] = X_transV[:,:dimTrans-2]
print(XV.shape)
XV[nTransV:,dimTrans-2:] = X_subV
print(XV.shape)
XV[:nTransV,-2:] = X_transV[:,-2:]
print(XV.shape)
yTr = np.zeros((nSubTr+nTransTr,1))
print(yTr.shape)
yTr[:nTransTr,0] = y_transCdTr.T
print(yTr.shape)
yTr[nTransTr:,0] = y_subCdTr.T
print(yTr.shape)
yV = np.zeros((nSubV+nTransV,1))
print(yV.shape)
yV[:nTransV,0] = y_transCdV.T
print(yV.shape)
yV[nTransV:,0] = y_subCdV.T
print(yV.shape)
dyTr = np.zeros((nTransTr+nSubTr,dimTrans + dimSub - 2))
print(dyTr.shape)
dyTr[:nTransTr,:dimTrans-2] = dy_transCdTr[:,:dimTrans-2]
print(dyTr.shape)
dyTr[nTransTr:,dimTrans-2:] = dy_subCdTr
print(dyTr.shape)
dyTr[:nTransTr,-2:] = dy_transCdTr[:,-2:]
print(dyTr.shape)
dyV = np.zeros((nTransV+nSubV,dimTrans + dimSub - 2))
print(dyV.shape)


(46159, 24)
(46159, 24)
(46159, 24)
(46159, 24)
(11722, 24)
(11722, 24)
(11722, 24)
(11722, 24)
(46159, 1)
(46159, 1)
(46159, 1)
(11722, 1)
(11722, 1)
(11722, 1)
(46159, 24)
(46159, 24)
(46159, 24)
(46159, 24)
(11722, 24)


In [11]:
# Set up NN parameters
INPUT_DIM = dimTrans+dimSub-2
OUTPUT_DIM = 1
NUM_SAMPLES =  int(XTr.shape[0]/3)
NUM_TRAINING = XTr.shape[0]
NUM_VALIDATING = XV.shape[0]
NUM_HIDDEN = 100
print(INPUT_DIM,OUTPUT_DIM,NUM_SAMPLES,NUM_TRAINING,NUM_VALIDATING,NUM_HIDDEN)

24 1 15386 46159 11722 100


In [12]:
class SobolevNetwork:
    def __init__(self, input_dim, num_hidden,init = None):
        self.input_dim = input_dim 
        self.num_hidden = num_hidden
        self.W1 = tf.Variable(tf.random_normal([self.input_dim, self.num_hidden],stddev=0.1))
        self.b1 = tf.Variable(tf.ones([self.num_hidden]))
        self.W2 = tf.Variable(tf.random_normal([self.num_hidden, self.num_hidden],stddev=0.1))
        self.b2 = tf.Variable(tf.ones([self.num_hidden]))
        self.W3 = tf.Variable(tf.random_normal([self.num_hidden, self.num_hidden],stddev=0.1))
        self.b3 = tf.Variable(tf.ones([self.num_hidden]))
        self.W4 = tf.Variable(tf.random_normal([self.num_hidden, self.num_hidden],stddev=0.1))
        self.b4 = tf.Variable(tf.ones([self.num_hidden]))
        self.W5 = tf.Variable(tf.random_normal([self.num_hidden, self.num_hidden],stddev=0.1))
        self.b5 = tf.Variable(tf.ones([self.num_hidden]))
        self.W6 = tf.Variable(tf.random_normal([self.num_hidden, self.num_hidden],stddev=0.1))
        self.b6 = tf.Variable(tf.ones([self.num_hidden]))        
        self.W7 = tf.Variable(tf.random_normal([self.num_hidden, 1],stddev=0.1))
        self.b7 = tf.Variable(tf.ones([1]))
        self.weights = [(self.W1, self.b1), (self.W2, self.b2), (self.W3, self.b3),(self.W4, self.b4), (self.W5, self.b5), (self.W6, self.b6),(self.W7, self.b7)]
        

    def forward(self, X):
        #Input layer
        out = X
        #Hidden layers
        W,b = self.weights[0]
        out = tf.nn.tanh(tf.matmul(out, W) + b)
        W,b = self.weights[1]
        out = tf.nn.tanh(tf.matmul(out, W) + b)
        W,b = self.weights[2]
        out = tf.nn.sigmoid(tf.matmul(out, W) + b)
        W,b = self.weights[3]
        out = tf.nn.sigmoid(tf.matmul(out, W) + b)
        W,b = self.weights[4]
        out = tf.nn.leaky_relu(tf.matmul(out, W) + b)
        W,b = self.weights[5]
        out = tf.nn.relu(tf.matmul(out, W) + b)
        #Output layer
        W,b = self.weights[-1]
        out = tf.matmul(out, W) + b
        return out

In [15]:
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior() 

Instructions for updating:
non-resource variables are not supported in the long term


In [16]:
# Define tensors and operations
X = tf.placeholder(tf.float32, shape=[None, INPUT_DIM],name='X')
y = tf.placeholder(tf.float32, shape=[None,OUTPUT_DIM],name='y')
y_der = tf.placeholder(tf.float32, shape=[None, INPUT_DIM],name='dydX')
print(X.shape,y.shape,y_der.shape)

(?, 24) (?, 1) (?, 24)


In [18]:
model = SobolevNetwork(INPUT_DIM, NUM_HIDDEN) 
y_p = model.forward(X)
predict_named = tf.identity(y_p, "prediction")
dy = tf.gradients(y_p, X)
predict_named = tf.identity(dy, "gradient")
optimizer = tf.train.AdamOptimizer()
print(model,y_p)

<__main__.SobolevNetwork object at 0x000001FDB0C58B80> Tensor("add_6:0", shape=(?, 1), dtype=float32)


In [19]:
predict_named

<tf.Tensor 'gradient:0' shape=(1, ?, 24) dtype=float32>

In [20]:
dy

[<tf.Tensor 'gradients/MatMul_grad/MatMul:0' shape=(?, 24) dtype=float32>]

In [21]:
predict_named

<tf.Tensor 'gradient:0' shape=(1, ?, 24) dtype=float32>

In [22]:
optimizer

In [23]:
yLossLambda = dict()
for lam in range(11):
    yLossLambda[lam] = tf.reduce_mean(tf.pow(y_p - tf.reshape(y,[NUM_SAMPLES,1]), 2) + lam/10. * tf.reshape(tf.reduce_sum(tf.pow(dy - y_der, 2),2),[NUM_SAMPLES,1]))


In [24]:
yLossLambda

{0: <tf.Tensor 'Mean:0' shape=() dtype=float32>,
 1: <tf.Tensor 'Mean_1:0' shape=() dtype=float32>,
 2: <tf.Tensor 'Mean_2:0' shape=() dtype=float32>,
 3: <tf.Tensor 'Mean_3:0' shape=() dtype=float32>,
 4: <tf.Tensor 'Mean_4:0' shape=() dtype=float32>,
 5: <tf.Tensor 'Mean_5:0' shape=() dtype=float32>,
 6: <tf.Tensor 'Mean_6:0' shape=() dtype=float32>,
 7: <tf.Tensor 'Mean_7:0' shape=() dtype=float32>,
 8: <tf.Tensor 'Mean_8:0' shape=() dtype=float32>,
 9: <tf.Tensor 'Mean_9:0' shape=() dtype=float32>,
 10: <tf.Tensor 'Mean_10:0' shape=() dtype=float32>}

In [25]:
# Train storage
optim = dict()
timer = dict()
resTrain = dict()
for lam in range(11):
    optim[lam] = optimizer.minimize(yLossLambda[lam])
    timer[lam] = []
    resTrain[lam] = []

In [26]:
# Validation storage
resValid = []
resValid1 = []
resValid2 = []

# Saved model storage
saver = tf.train.Saver(max_to_keep=100)
errModel = np.zeros((1000,1000))

In [27]:
saver,errModel

(<tensorflow.python.training.saver.Saver at 0x1fdb4c0d4c0>,
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]))

In [28]:
# Sobolev optimizer
tstart = time.time()
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [29]:
tstart,sess,init,sess

(1667414331.7105114,
 <tf.Operation 'init' type=NoOp>,
 <tensorflow.python.client.session.Session at 0x1fda3a1d0d0>)

In [30]:
X_valid = XV;y_valid = yV;dy_valid = dyV
valid_dict = {X: X_valid, y: y_valid, y_der: dy_valid}    
X_valid1 = XV[:nTransV,:];y_valid1 = yV[:nTransV,:];dy_valid1 = dyV[:nTransV,:]
valid_dict1 = {X: X_valid1, y: y_valid1, y_der: dy_valid1}
X_valid2 = XV[nTransV:,:];y_valid2 = yV[nTransV:,:];dy_valid2 = dyV[nTransV:,:]
valid_dict2 = {X: X_valid2, y: y_valid2, y_der: dy_valid2}
best_sub = 1000
best_trans = 1000
iter = 0

In [35]:
X_valid.shape

(11722, 24)

In [36]:
batch_samples = random.sample(range(NUM_TRAINING),NUM_SAMPLES)
X_train = XTr[batch_samples,:]
y_train = yTr[batch_samples,:]
dy_train = dyTr[batch_samples,:]
train_dict = {X: X_train, y: y_train, y_der: dy_train}

In [40]:
X_train.shape,y_train.shape,dy_train.shape

((15386, 24), (15386, 1), (15386, 24))